## Framework to setup the class and the board

In [ ]:
import numpy as np
import pandas as pd
from __future__ import annotations
from re import sub
from typing import Dict, List, Optional, Set, Tuple
# I am using Pandas even though we can do this without because it is useful to get used to it.

class Sudoku(object):
    
    def __init__(self,board=False,test=False):
        
        self.test = test
        if test:
            # If this is a test, I ignore your board
            self.board = self.test_board(test)
        elif not board :
            # I wasn't given a board, so I will assume Test0
            self.board = self.test_board(0)
        else:
            self.board = board
        self.set_lists = self.initialize_sets()
    
    def initialize_sets(self):
        set_dict = {}
        set_list = [set(range(9))]*9
        for col in ['A','B','C','D','E','F','G','H','I']:
            set_dict[col] = set_list
        return set_dict
    
    def violation_check(self,board=False):
        # You can pass a board if you want to look ahead
        # if you dont pass a board, it will use self.board
        # This might not be how you want to check if a move is good
        # I am using a technique of checking a board after the addition.
        # That is not efficient, you might want to just look at the addition to see if it causes an issue
        # I am using constraint check mostly for the grading script
        if isinstance(board,bool):
            board = self.board
        
        cols = ['A','B','C','D','E','F','G','H','I']
        
        for ind in board.index:
            row_series = board.loc[ind]
            test_row = row_series.dropna()
            for item in test_row.duplicated():
                 if item:
                    print("Row Violation")
                    return True
                
        for col in cols:
                col_series = board[col]
                test_col = col_series.dropna()
                for item in test_col.duplicated():
                    if item:
                        print("Column Violation")
                        return True
            # The more annoying case is box. 
            #I am sure I can come up with a clean way to check box, but not today
            # After I wrote this I realized I could have just used bounds of box and looked using that
        boxes = [{('A', 0), ('A', 1), ('A', 2),('B', 0), ('B', 1), ('B', 2),('C', 0), ('C', 1), ('C', 2)},
                 {('A', 3), ('A', 4), ('A', 5),('B', 3), ('B', 4), ('B', 5),('C', 3), ('C', 4), ('C', 5)},
                 {('A', 6), ('A', 7), ('A', 8),('B', 6), ('B', 7), ('B', 8),('C', 6), ('C', 7), ('C', 8)},
                 {('D', 0), ('D', 1), ('D', 2),('E', 0), ('E', 1), ('E', 2),('F', 0), ('F', 1), ('F', 2)},
                 {('D', 3), ('D', 4), ('D', 5),('E', 3), ('E', 4), ('E', 5),('F', 3), ('F', 4), ('F', 5)},
                 {('D', 6), ('D', 7), ('D', 8),('E', 6), ('E', 7), ('E', 8),('F', 6), ('F', 7), ('F', 8)},
                 {('G', 0), ('G', 1), ('G', 2),('H', 0), ('H', 1), ('H', 2),('I', 0), ('I', 1), ('I', 2)},
                 {('G', 3), ('G', 4), ('G', 5),('H', 3), ('H', 4), ('H', 5),('I', 3), ('I', 4), ('I', 5)},
                 {('G', 6), ('G', 7), ('G', 8),('H', 6), ('H', 7), ('H', 8),('I', 6), ('I', 7), ('I', 8)}  
            ]
            
        # Yes, I know. Bad complexity
        for box in boxes:
            for cell1 in box:
                 for cell2 in box:
                    if cell1 != cell2:
                        if board[cell1[0]][cell1[1]] == board[cell2[0]][cell2[1]]:
                            print("Box violation")
                            return True
        return False          
                           
    def already_solved(self,good=True) :
        # The purpose of this is to do a quick check on the grading script
        board_dict = {}
        if good:
            board_dict = {'A':[5,7,3,4,2,1,8,9,6],
             'B':[8,9,6,3,5,7,4,1,2],
             'C':[1,2,4,8,6,9,5,3,7],
             'D':[6,8,5,9,1,3,2,7,4],
             'E':[7,4,9,5,8,2,1,6,3],
             'F':[2,3,1,7,4,6,9,8,5],
             'G':[4,6,7,2,9,8,3,5,1],
             'H':[3,5,8,1,7,4,6,2,9],
             'I':[9,1,2,6,3,5,7,4,8]}
        else:
            board_dict = {'A':[5,7,3,4,2,1,8,9,6],
             'B':[5,9,6,3,8,7,4,1,2],
             'C':[1,2,4,8,6,9,5,3,7],
             'D':[6,8,5,9,1,3,2,7,4],
             'E':[7,4,9,5,8,2,1,6,3],
             'F':[2,3,1,7,4,6,9,8,5],
             'G':[4,6,7,2,9,8,3,5,1],
             'H':[3,5,8,1,7,4,6,2,9],
             'I':[9,1,2,6,3,5,7,4,8]}

        return pd.DataFrame(data=board_dict)
            
    def test_board(self,index=0):
        board_dict = {}
        if index == 0: # Easy
            board_dict= {'A':[3,np.nan,1,7,np.nan,np.nan,5,np.nan,np.nan],'B':[6,4,np.nan,np.nan,np.nan,8,np.nan,3,np.nan],'C':[5,np.nan,7,2,np.nan,3,np.nan,np.nan,np.nan],
                       'D':[np.nan,np.nan,np.nan,np.nan,6,np.nan,8,np.nan,5],'E':[9,6,5,8,np.nan,np.nan,np.nan,7,4],'F':[2,1,8,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan],
                       'G':[7,np.nan,9,np.nan,np.nan,4,6,np.nan,1],'H':[np.nan,np.nan,6,np.nan,np.nan,7,2,np.nan,np.nan],'I':[8,np.nan,4,np.nan,2,np.nan,np.nan,np.nan,7]}
        elif index == 1: # Medium
            board_dict= {'A':[np.nan,8,7,np.nan,np.nan,np.nan,4,2,np.nan],'B':[6,np.nan,np.nan,np.nan,np.nan,7,np.nan,3,np.nan],'C':[np.nan,np.nan,np.nan,np.nan,np.nan,8,np.nan,np.nan,np.nan],
                       'D':[3,7,np.nan,np.nan,8,np.nan,np.nan,np.nan,1],'E':[np.nan,9,np.nan,np.nan,6,np.nan,np.nan,4,np.nan],'F':[4,np.nan,1,np.nan,7,np.nan,np.nan,5,2],
                       'G':[7,np.nan,np.nan,np.nan,3,np.nan,6,np.nan,4],'H':[np.nan,4,np.nan,np.nan,5,np.nan,2,np.nan,np.nan],'I':[np.nan,2,np.nan,7,4,6,np.nan,np.nan,5]}
            
        elif index == 2: # Hard
            board_dict = {'A':[np.nan,np.nan,np.nan,np.nan,np.nan,5,np.nan,np.nan,2],'B':[1,np.nan,np.nan,7,3,np.nan,np.nan,9,np.nan],'C':[np.nan,np.nan,3,np.nan,np.nan,9,7,6,np.nan],
                       'D':[np.nan,5,4,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan],'E':[np.nan,np.nan,np.nan,4,np.nan,3,np.nan,np.nan,np.nan],'F':[np.nan,6,2,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan],
                       'G':[np.nan,np.nan,np.nan,3,4,np.nan,np.nan,np.nan,5],'H':[9,np.nan,1,np.nan,6,8,np.nan,2,np.nan],'I':[np.nan,np.nan,np.nan,1,np.nan,np.nan,np.nan,np.nan,np.nan]}
        elif index == 99: # Bad board D4 = I4 
            board_dict = {'A':[np.nan,np.nan,np.nan,np.nan,np.nan,5,np.nan,np.nan,2],'B':[1,np.nan,np.nan,7,3,np.nan,np.nan,9,np.nan],'C':[np.nan,np.nan,3,np.nan,np.nan,9,7,6,np.nan],
                       'D':[np.nan,5,4,np.nan,2,np.nan,np.nan,np.nan,np.nan],'E':[np.nan,np.nan,np.nan,4,np.nan,3,np.nan,np.nan,np.nan],'F':[np.nan,6,2,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan],
                       'G':[np.nan,np.nan,np.nan,3,4,np.nan,np.nan,np.nan,5],'H':[9,np.nan,1,np.nan,6,8,np.nan,2,np.nan],'I':[np.nan,np.nan,np.nan,1,2,np.nan,np.nan,np.nan,np.nan]}
        elif index == 98: # Bad board  I4 = H3
            board_dict = {'A':[np.nan,np.nan,np.nan,np.nan,np.nan,5,np.nan,np.nan,2],'B':[1,np.nan,np.nan,7,3,np.nan,np.nan,9,np.nan],'C':[np.nan,np.nan,3,np.nan,np.nan,9,7,6,np.nan],
                       'D':[np.nan,5,4,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan],'E':[np.nan,np.nan,np.nan,4,np.nan,3,np.nan,np.nan,np.nan],'F':[np.nan,6,2,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan],
                       'G':[np.nan,np.nan,np.nan,3,4,np.nan,np.nan,np.nan,5],'H':[9,np.nan,1,2,6,8,np.nan,np.nan,np.nan],'I':[np.nan,np.nan,np.nan,1,2,np.nan,np.nan,np.nan,np.nan]}
        
        else: # Expert
            board_dict = {'A':[np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan],'B':[1,9,np.nan,np.nan,8,np.nan,3,np.nan,np.nan],'C':[np.nan,6,np.nan,np.nan,5,4,np.nan,9,np.nan],
                       'D':[np.nan,np.nan,2,np.nan,np.nan,np.nan,np.nan,np.nan,5],'E':[np.nan,np.nan,np.nan,np.nan,6,np.nan,np.nan,3,4],'F':[np.nan,7,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan],
                       'G':[5,np.nan,np.nan,8,np.nan,np.nan,np.nan,np.nan,np.nan],'H':[np.nan,np.nan,1,np.nan,np.nan,np.nan,9,np.nan,6],'I':[4,np.nan,np.nan,7,2,np.nan,np.nan,5,np.nan]}
            
        return pd.DataFrame(data=board_dict)


## Solving Sudoku
I set np.nan as a value to indicate an empty spot on the board. The test will check that there are not any np.nan and that the board is consistent. 


In [5]:
class Solution(Sudoku):
    def __init__(self, n=0, symbols=[], symbol_set={}, board=False, test=False, grid_map=None):

        assert n == len(symbols)
        self.test = test
        super().__init__(board,test)
        self._n, self._symbols, self._symbol_set, self._set_map = n, symbols, symbol_set,{}
        if grid_map is None:
            self._map = {}
            self._populating_fun()
        else:
            self._map = grid_map
    
    ## Your code ###

    def __str__(self):
        """
        Returns an easily readable string representation of the current puzzle.
        """
        string_repr, n = [], round(self._n ** (1 / 2))
        for i in range(self._n):
            row_lst = self._symbols[i][:]
            string_repr.append(' '.join(row_lst))
        return '\n'.join(string_repr)
    
    def _populating_fun(self):
        """
        This function updates _map with possible symbols for each unfilled position in the sudoku puzzle.
        """
        for r in range(self._n):
            for c in range(self._n):
                if self._symbols[r][c] == '*':
                    subset = self._row_set(r) | self._column_set(c) | self._subsquare_set(r, c)
                    allowed_symbols = self._symbol_set - subset
                    self._map[(r, c)] = allowed_symbols

    def _populating_setting(self):
        """
        This function updates _set_map with missing symbols for each set and the positions they could possibly 
        occupy within the set.
        """
        for r in range(self._n):
            set_name = f'row{r}'
            self._set_map[set_name] = {}
            row_set = self._row_set(r)
            missing_symbols = self._symbol_set - row_set
            for sym in missing_symbols:
                self._set_map[set_name][sym] = set()
                for key, value in self._map.items():
                    if key[0] == r and sym in value:
                        self._set_map[set_name][sym].add(key)
        if self._n > 9:
            # these computations take up too much time for the regular 9x9 puzzles
            for c in range(self._n):
                set_name = f'col{c}'
                self._set_map[set_name] = {}
                col_set = self._column_set(c)
                missing_symbols = self._symbol_set - col_set
                for sym in missing_symbols:
                    self._set_map[set_name][sym] = set()
                    for key, value in self._map.items():
                        if key[1] == c and sym in value:
                            self._set_map[set_name][sym].add(key)
            n = round(self._n ** (1 / 2))
            for r in range(0, self._n, n):
                for c in range(0, self._n, n):
                    set_name = f'ss{r // n}{c // n}'
                    self._set_map[set_name] = {}
                    subsq_set = self._subsquare_set(r, c)
                    missing_symbols = self._symbol_set - subsq_set
                    for sym in missing_symbols:
                        self._set_map[set_name][sym] = set()
                        for key, value in self._map.items():
                            if key[0] // n == r // n and key[1] // n == c // n \
                                    and sym in value:
                                self._set_map[set_name][sym].add(key)

    def get_symbols(self):
        """
        This function returns a copy of symbols for use during testing. Here symbols are numbers from 1 to 9 in 
        string format.
        """
        return [row[:] for row in self._symbols]

    def is_solved(self):
        """
        Returns whether the current puzzle is solved.
        """
        return not any('*' in row for row in self._symbols) \
            and self._check_row_and_col() and self._check_subsquares()
    
    def _check_row_and_col(self):
        # (helper for is_solved)
        # checks that all rows and columns are filled in properly
        return all(self._row_set(i) == self._symbol_set and
                   self._column_set(i) == self._symbol_set
                   for i in range(self._n))

    def _check_subsquares(self):
        # (helper for is_solved)
        # checks that all subsquares are filled in properly
        n = round(self._n ** (1 / 2))
        return all(self._subsquare_set(r, c) == self._symbol_set
                   for r in range(0, self._n, n) for c in range(0, self._n, n))
    
    

    def depth_first_solve(self,puzzle):
        """
        An iterative depth first search to solve the puzzle.
        """
        if puzzle.is_solved():
            return puzzle
        puzzle_queue = puzzle.extensions()
        while puzzle_queue:
            curr = puzzle_queue.pop()
            if curr.is_solved():
                return curr
            puzzle_queue.extend(curr.extensions())
        return None
    
    def extensions(self):
        """
        Returns a list of SudokuPuzzle objects that have the position
        with the least number of possibilities filled in.
        This method checks for naked singles first, and if none are found,
        checks for hidden singles. Again, if none are found, it fills in the
        spot with the least number of naked/hidden possibilities.
        """
        if not self._map:
            return []
        extensions = []
        position, possible = None, self._symbol_set | {'*'}
        for pos, values in self._map.items():
            if len(values) < len(possible):
                position, possible = pos, values
        symbol, possible_positions = None, None
        if len(possible) > 1:
            self._populating_setting()
            for d in self._set_map.values():
                for sym, positions in d.items():
                    if len(positions) < len(possible):
                        symbol, possible_positions, = sym, positions
        if symbol:
            for pos in possible_positions:
                new_symbols = [row[:] for row in self._symbols]
                new_symbols[pos[0]][pos[1]] = symbol
                new_map = self._map.copy()
                for key in self._get_positions(pos):
                    new_map[key] = self._map[key] - {symbol}
                del new_map[pos]
                extensions.append(Solution(self._n, new_symbols,
                                               self._symbol_set, new_map))
        else:
            for value in possible:
                new_symbols = [row[:] for row in self._symbols]
                new_symbols[position[0]][position[1]] = value
                new_map = self._map.copy()
                for key in self._get_positions(position):
                    new_map[key] = self._map[key] - {value}
                del new_map[position]
                extensions.append(Solution(self._n, new_symbols, self._symbol_set, new_map))
        return extensions

    def _get_positions(self, pos):
        """
        This function returns the keys of sets in _map that may need to be altered.
        """
        n = round(self._n ** (1 / 2))
        return [key for key in self._map if key[0] == pos[0] or key[1] == pos[1] or (key[0] // n == pos[0] // n and
                key[1] // n == pos[1] // n)]

    def _row_set(self, r):
        """
        This function returns the set of symbols of row r.
        """
        return set(self._symbols[r])

    def _column_set(self, c):
        """
        This function returns the set of symbols of column c.
        """
        return set(row[c] for row in self._symbols)

    def _subsquare_set(self, r, c):
        # returns the set of symbols of the subsquare that [r][c] belongs to
        n, symbols = self._n, self._symbols
        ss = round(n ** (1 / 2))
        ul_row = (r // ss) * ss
        ul_col = (c // ss) * ss
        return set(symbols[ul_row + i][ul_col + j]
                   for i in range(ss) for j in range(ss))
    
    def is_valid_grid(self,lst, symbol_set):
        return not any(lst[r].count(sym) > 1 or
                   [row[c] for row in lst].count(sym) > 1
                   or self._sbsq_lst(r, c, lst).count(sym) > 1
                   for r in range(len(lst)) for c in range(len(lst[r]))
                   for sym in symbol_set)

    def _sbsq_lst(self,r, c, symbols):
        # (helper for is_valid_grid)
        # returns the list of symbols in the subsquare containing [r][c]
        ss = round(len(symbols) ** (1 / 2))
        return [symbols[(r // ss) * ss + i][(c // ss) * ss + j]
                for i in range(ss) for j in range(ss)]
    
    def make_9x9_sudoku_puzzle(self,x):

        symbol_set = {str(i) for i in range(1, 10)}
        obj = Sudoku()
        df = obj.test_board(index=x)
        df.fillna('*',inplace=True)

        for i in ['A','B','C','D','E','F','G','H','I']:
            for j in range(9): 
                if df[i][j]!='*':
                    df[i][j] = str(int(df[i][j]))

        symbols = df.values.tolist()

        if self.is_valid_grid(symbols, symbol_set):
            return Solution(9, symbols, symbol_set)
        else:
            return 0
    
    def solution(self):

        ### Your code here

        a = self.make_9x9_sudoku_puzzle(self.test)
        if a!=0:
            sol = self.depth_first_solve(a)
            j = 0
            A = []
            while(j<len(str(sol))):
                d = [int(i) for i in str(sol)[j:j+18:2]]
                j = j+18
                A.append(d)
            df2 = pd.DataFrame(A,columns=['A','B','C','D','E','F','G','H','I'])
            return df2
        else:
            print ("\nInvalid Puzzle. Please Try Again with Valid Sudoku Puzzle.")

    

## Testing` Script

In [6]:
### Grading script
tests = 4
correct_result = 0
for i in range(tests):
    test = Solution(test=i)
    sol_board = test.solution()
    print(sol_board)
    if sol_board.isnull().sum().sum() > 0:
        print("Board",i,"is not solved")
    elif not test.violation_check(sol_board):     
        print("Board",i,"looks ok from a quick check")
        correct_result +=1
    else:
        print("Board",i,"is inconsistent")
print("The script indicates that you got {} out of {} correct".format(correct_result,tests))
print("Score is {:.0f}".format(correct_result/tests*100))

   A  B  C  D  E  F  G  H  I
0  3  6  5  4  9  2  7  1  8
1  9  4  8  7  6  1  2  3  5
2  1  2  7  3  5  8  9  6  4
3  7  5  2  1  8  4  3  9  6
4  4  1  9  6  3  7  8  5  2
5  6  8  3  9  2  5  4  7  1
6  5  7  4  8  1  9  6  2  3
7  8  3  1  2  7  6  5  4  9
8  2  9  6  5  4  3  1  8  7
Board 0 looks ok from a quick check
   A  B  C  D  E  F  G  H  I
0  9  6  5  3  2  4  7  1  8
1  8  1  3  7  9  6  5  4  2
2  7  2  4  5  8  1  9  6  3
3  3  4  6  2  5  9  1  8  7
4  1  9  2  8  6  7  3  5  4
5  5  7  8  4  1  3  2  9  6
6  4  5  7  9  3  8  6  2  1
7  2  3  1  6  4  5  8  7  9
8  6  8  9  1  7  2  4  3  5
Board 1 looks ok from a quick check
   A  B  C  D  E  F  G  H  I
0  4  1  5  3  8  7  6  9  2
1  9  2  8  5  1  6  7  3  4
2  7  6  3  4  9  2  8  1  5
3  6  7  2  9  4  8  3  5  1
4  8  3  1  2  7  5  4  6  9
5  5  4  9  6  3  1  2  8  7
6  1  5  7  8  2  3  9  4  6
7  3  9  6  7  5  4  1  2  8
8  2  8  4  1  6  9  5  7  3
Board 2 looks ok from a quick check
   A  B  C  D  E  F  G